In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from matplotlib import style
from io import StringIO
from pandas import read_csv
from sklearn.model_selection import train_test_split
from  IPython import display
import pathlib
import shutil
import tempfile
from sklearn.preprocessing import PowerTransformer
import scipy.stats
from scipy.stats.stats import pearsonr
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import TransformerMixin 
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error

In [16]:
column_names = ['paper','Ti','Nb','Hf','Zr', 'Al','Ta', 'Mo','W','V', 
                 'Cr', 'Si', 'plas','homT', 'homt','ar', 'ardiff','Vec', 'target', 'temperature','Tm','Hmix', 'Smix'] # 'temperature', 'Tm',
dataframe = pd.read_csv("HT Strength.csv", names=column_names, header=None, sep=';', decimal='.')
dataset = dataframe.values
np.random.shuffle(dataset)

In [17]:
Nb = dataframe["Nb"]
Ti =  dataframe["Ti"]
Hf = dataframe["Hf"]
Zr = dataframe["Zr"]
Al = dataframe["Al"]
Ta = dataframe["Ta"]
Mo = dataframe["Mo"]
W = dataframe["W"]
V = dataframe["V"]
Cr = dataframe["Cr"]
Si = dataframe["Si"]
plas = dataframe["plas"]
homT = dataframe["homT"]
homt = dataframe["homt"]
ar = dataframe["ar"]
ardiff = dataframe["ardiff"]
Vec = dataframe["Vec"]
paper = dataframe["paper"]
target = dataframe["target"]
temperature = dataframe['temperature']
Tm = dataframe["Tm"]
Hmix = dataframe["Hmix"]
Smix = dataframe["Smix"]

In [20]:
dataframe2 =  pd.concat([ar, plas, homT, homt, ardiff, Vec, Hmix, Smix, temperature, Tm, target], axis=1) # temperature, Tm,
dataset2 = dataframe2.values #Ti, Nb, Hf, Zr, Al, Ta, Mo, W, V, Cr, Si, 

df=pd.DataFrame(dataset2)
df.shape

(370, 11)

In [22]:
pt=PowerTransformer(method='yeo-johnson', standardize=True)
pt.fit(df)
data_train2=pd.DataFrame(pt.transform(df))


In [52]:
svr_score = 0.1
svr_scoretrain = 0.1
i=0
   
data_train, data_test= train_test_split(df, test_size=0.20, shuffle=True)

data_train=pd.DataFrame(data_train)
data_test=pd.DataFrame(data_test)
pt=PowerTransformer(method='yeo-johnson', standardize=True)
pt.fit(data_train)
data_train2=pt.transform(data_train)
data_test2=pt.transform(data_test)
x_train = data_train2[:,0:-1]
x_test = data_test2[:,0:-1]
y_train = data_train2[:,-1] 
y_test = data_test2[:,-1]


regressor = SVR(kernel = 'rbf', tol=1e-7, C=32, epsilon=0.05)  

regressor.fit(x_train, y_train)
y_predtest = regressor.predict(x_test)
y_predtrain = regressor.predict(x_train)
svr_score = regressor.score(x_test,y_test)
svr_scoretrain = regressor.score(x_train,y_train)


print(svr_score)  
regressor.score(x_train,y_train)   

0.9325231712257819


0.9524368317395154

In [53]:
y_predictions_test=pd.DataFrame(y_predtest)
y_predictions_train=pd.DataFrame(y_predtrain)
a=pd.DataFrame(y_test)
b=pd.DataFrame(x_test)
c=pd.DataFrame(y_train)
d=pd.DataFrame(x_train)
svm_comparison_test=pd.concat([b, a], axis=1)
svm_result_test=pd.concat([b, y_predictions_test], axis=1)
svm_comparison_train=pd.concat([d, c], axis=1)
svm_result_train=pd.concat([d, y_predictions_train], axis=1)

#undo: power transform
svm_result_testuntrans=pt.inverse_transform(svm_result_test)
svm_comparison_testuntrans=pt.inverse_transform(svm_comparison_test)
svm_result_trainuntrans=pt.inverse_transform(svm_result_train)
svm_comparison_trainuntrans=pt.inverse_transform(svm_comparison_train)


output_test_untrans = svm_result_testuntrans[:,-1]
target_test_untrans = svm_comparison_testuntrans[:,-1]
output_train_untrans = svm_result_trainuntrans[:,-1]
target_train_untrans = svm_comparison_trainuntrans[:,-1]
loss_test= mean_squared_error(target_test_untrans,output_test_untrans)
loss_train=mean_squared_error(target_train_untrans,output_train_untrans)


aysu=np.mean(loss_test)
mert=np.mean(loss_train)
print(np.sqrt(aysu))  #test rmse
np.sqrt(mert)   #train rmse


136.02391140930808


103.52101694732598

In [57]:
A =  svm_comparison_testuntrans[:,-1]
C =  svm_comparison_trainuntrans[:,-1]
svm_result_testuntrans2 = pd.DataFrame(svm_result_testuntrans)
svm_result_trainuntrans2 = pd.DataFrame(svm_result_trainuntrans)
A2 = pd.DataFrame(A)
C2 = pd.DataFrame(C)
svm_resultcomp_testuntrans = pd.concat([svm_result_testuntrans2, A2], axis=1)
svm_resultcomp_trainuntrans = pd.concat([svm_result_trainuntrans2, C2], axis=1)
svm_result = pd.concat([svm_resultcomp_testuntrans, svm_resultcomp_trainuntrans], axis=0)

svm_result

,0,1,2,3,4,5,6,7,8,9,10,0
0,148.296000,1400.0,0.0,0.0,4.225608,4.666000,2.966400,9.119860,1000.0,24.581770,364.322041,319.0
1,140.333334,0.0,0.0,0.0,4.065853,5.166667,-4.222222,14.896688,600.0,25.195000,932.058194,973.0
2,143.308000,0.0,1400.0,35.0,2.915520,5.032000,-4.845952,11.514903,600.0,26.721800,586.495656,595.0
3,145.628000,0.0,1400.0,35.0,1.303902,4.660000,1.364080,9.128775,100.0,23.731200,406.213087,447.0
4,150.095238,0.0,0.0,0.0,4.697146,4.476190,1.560091,14.335349,25.0,22.666190,1150.657782,1112.0
...,...,...,...,...,...,...,...,...,...,...,...,...
291,146.000000,0.0,0.0,0.0,5.458006,4.750000,-2.500000,11.525651,25.0,21.557500,1559.033215,1591.0
292,145.750000,0.0,0.0,0.0,6.201107,4.625000,-0.875000,12.966358,20.0,21.596250,1289.760944,1260.0
293,142.333333,0.0,0.0,0.0,4.069824,4.555556,-8.395062,13.145213,25.0,20.893667,1032.303504,1012.0
294,148.692000,1200.0,1200.0,24.0,3.879119,4.354000,-3.966104,12.585219,25.0,20.867201,1930.651374,1965.0


In [58]:
svm_result.to_csv (r'C:\****.csv', index = False, header=True,sep=',', decimal='.')